In [1]:
import numpy as np
import pandas as pd
import re
import tabula
import pdfplumber
import pandas as pd
from collections import namedtuple
from PyPDF2 import PdfReader
import glob
import os
from tqdm.auto import tqdm
from tkinter import filedialog as fd
import json
from pathlib import Path


In [2]:
from PyPDF2 import PdfFileMerger
from datetime import date

In [3]:
today = date.today()
today_string = str(today)

In [4]:
merger = PdfFileMerger()

In [5]:
file_name = "file_name_in"
file_name_out = file_name + today_string + ".pdf"
directory = 'directory_path_out'
path = os.path.join(directory, file_name_out)

merger = PdfFileMerger()


path_to_files = r'path_where_the_files_are_located/' #Note that you must insert the "/" 


for root, dirs, file_names in os.walk(path_to_files):
    
    for file_name in file_names:
        
        merger.append(path_to_files + file_name)
merger.write(path)
merger.close()    

In [7]:
Line = namedtuple('Line', ' UC Itens_de_Fatura Numeros')

In [8]:


en_pt_mpt_tusd_re = re.compile(r'(ENERGIA INJ. PT OUC MPT TUSD) (\d{2}/\d{4})(kWh) (.*) (.*) (.*)') 
en_in_pt_mpt_te = re.compile(r'(ENERGIA INJ. PT OUC MPT TE) (\d{2}/\d{4}) (kWh) (.*) (.*) (.*)')


en_fp_mpt_tusd_re = re.compile(r'(ENERGIA INJ. FP OUC MPT TUSD) (\d{2}/\d{4})(kWh) (.*) (.*) (.*)') 
en_fp_mpt_te_re = re.compile(r'(ENERGIA INJ. FP OUC MPT TE) (\d{2}/\d{4}) (kWh) (.*) (.*) (.*)') 


en_pt_opt_tusd_re = re.compile(r'(ENERGIA INJ. PT OUC OPT TUSD) (\d{2}/\d{4})(kWh) (.*) (.*) (.*)') 
en_pt_opt_te_re = re.compile(r'(ENERGIA INJ. PT OUC OPT TE) (\d{2}/\d{4}) (kWh) (.*) (.*) (.*)')

ener_inj_ouc_mpt_te_re = re.compile(r'(ENERGIA INJ. OUC MPT TE) (\d{2}/\d{4}) (kWh) (.*) (.*) (.*)') 
ENERGIA_INJ_OUC_MPT_TUSD_re = re.compile(r'(ENERGIA INJ. OUC MPT TUSD) (\d{2}/\d{4}) (kWh) (.*) (.*) (.*)') 

ENERGIA_INJ_OUC_OPT_TE = re.compile(r'(ENERGIA INJ. OUC OPT TE) (\d{2}/\d{4}) (kWh) (.*) (.*) (.*)') 
ENERGIA_INJ_OUC_OPT_TUSD = re.compile(r'(ENERGIA INJ. OUC OPT TUSD) (\d{2}/\d{4}) (kWh) (.*) (.*) (.*)') 

uc_re = re.compile(r'(UC )(:) (\d+)') 

In [10]:
lines = []
with pdfplumber.open(path) as pdf:
    pages = pdf.pages
    for page in pages:
        text = page.extract_text()
        for row in text.split('\n'):
            
            if uc_re.findall(row):
                uc = uc_re.search(row).group(3)
                
            

            elif en_pt_mpt_tusd_re.findall(row):
                itens_fat, nums = en_pt_mpt_tusd_re.search(row).group(1), en_pt_mpt_tusd_re.search(row).group(4)
                lines.append(Line(uc, itens_fat, nums))
            
            elif en_in_pt_mpt_te.findall(row):
                itens_fat, nums = en_in_pt_mpt_te.search(row).group(1), en_in_pt_mpt_te.search(row).group(4)
                lines.append(Line(uc, itens_fat, nums))
            
            elif en_fp_mpt_tusd_re.findall(row):
                itens_fat, nums = en_fp_mpt_tusd_re.search(row).group(1), en_fp_mpt_tusd_re.search(row).group(4)
                lines.append(Line(uc, itens_fat, nums))
            
            elif en_fp_mpt_te_re.findall(row):
                itens_fat, nums = en_fp_mpt_te_re.search(row).group(1), en_fp_mpt_te_re.search(row).group(4)
                lines.append(Line(uc, itens_fat, nums))
            
            elif en_pt_opt_tusd_re.findall(row):
                itens_fat, nums = en_pt_opt_tusd_re.search(row).group(1), en_pt_opt_tusd_re.search(row).group(4)
                lines.append(Line(uc, itens_fat, nums))
                
            elif en_pt_opt_te_re.findall(row):
                itens_fat, nums = en_pt_opt_te_re.search(row).group(1), en_pt_opt_te_re.search(row).group(4)
                lines.append(Line(uc, itens_fat, nums))
                
            elif ener_inj_ouc_mpt_te_re.findall(row):
                itens_fat, nums = ener_inj_ouc_mpt_te_re.search(row).group(1), ener_inj_ouc_mpt_te_re.search(row).group(4)
                lines.append(Line(uc, itens_fat, nums))
                
            elif ENERGIA_INJ_OUC_MPT_TUSD_re.findall(row):
                itens_fat, nums = ENERGIA_INJ_OUC_MPT_TUSD_re.search(row).group(1), ENERGIA_INJ_OUC_MPT_TUSD_re.search(row).group(4)
                lines.append(Line(uc, itens_fat, nums))    
                
            elif ENERGIA_INJ_OUC_OPT_TE.findall(row):
                itens_fat, nums = ENERGIA_INJ_OUC_OPT_TE.search(row).group(1), ENERGIA_INJ_OUC_OPT_TE.search(row).group(4)
                lines.append(Line(uc, itens_fat, nums))
                
            elif ENERGIA_INJ_OUC_OPT_TUSD.findall(row):
                itens_fat, nums = ENERGIA_INJ_OUC_OPT_TUSD.search(row).group(1), ENERGIA_INJ_OUC_OPT_TUSD.search(row).group(4)
                lines.append(Line(uc, itens_fat, nums))    
            
            
                
                
                
                
                


In [ ]:
dfl = pd.DataFrame(lines)


In [12]:
dfl.to_csv(os.path.join(directory,file_name + ' - Parsed - ' + today_string),index=False)